# Getting Started with Jupyter and FPGA-SPICE on Linux

1. Install Anaconda: https://conda.io/docs/user-guide/install/linux.html 
2. Run anaconda-navigator: `~/anaconda-navigator`
3. Launch jupyter notebook from anaconda navigator

In the jupyter interface, navigate to the location of a notebook and view it.

The repository contains a submodule, so you need to get it as well:

Build the VPR tool with the FPGA-SPICE functionality:

In [38]:
import os
# move to the directory with the Makefile
os.chdir("./tangxifan-eda-tools/branches/vpr7_rram/vpr/")
# run make
print ("Wait for make to finish with a return code of 0.")
os.system("make")

Wait for make to finish with a return code of 0.


0

To run FPGA-SPICE, define its location:

In [39]:
# set this to the location of the vpr executable file
fpga_spice = "~/OpenFPGA/tangxifan-eda-tools/branches/vpr7_rram/vpr/vpr"

Define an architecture file and a circuit file:

In [40]:
arch = "./tutorial/example_arch.xml"
circuit = "./tutorial/example_circuit.blif"

Run vpr:

In [41]:

command_line = fpga_spice + " " + arch + " " + circuit
os.system(command_line)

256

This will bring up a graphical display of how the circuit is being placed on the FPGA. Press the `Proceed` button to step to the final placement, press `Proceed` again to step to the routing. Press the `Exit` button to exit the display.

To run VPR without the display, use the command `-nodisp`

In [23]:
command_line_nodisp = command_line + " -nodisp"


In [36]:
os.system(command_line_nodisp)

0

When you ran the tool, a number of text output files are created that report on what the tool did. Open the file "OpenFPGA/tutorial/example_circuit.place" in a text editor and note on line 2: 

> `Array size: 2 x 2 logic blocks`

This is a report that when the tool placed and routed the logic, it ended up in a 2 x 2 grid.

Now open the file "OpenFPGA/tutorial/example_arch.xml" in a text editor. Go to line 51 and change:

<pb_type name="io" capacity="1">

to:

<pb_type name="io" capacity="3">

Now, instead of each input/output block only having 1 input or output, they can have 3 inputs or outputs.

Run the tool again:

In [34]:
os.system(command_line_nodisp)

0

Re-open the file "OpenFPGA/tutorial/example_circuit.place" in a text editor and note that the logic is now placed in one block:

> `Array size: 1 x 1 logic blocks`

The simple change to the number of inputs or outputs per I/O block let the placement and routing tool use less space on the FPGA for the circuit.

# The Architecture format
An FPGA architecture is specified in an XML file and is wrapped in an `<architecture>` tag. `example_arch.xml` defines a simple FPGA.

Line 18 has the `<models>` tag which would describe `BLIF` circuit model names that the FPGA accepts. This architecture is simple enough to not need any additional models beyond the default `.names .latch .input .output`.

## Layout and Routing

Line 23 has the `<layout>` tag which specifies how the FPGA grid will be laid out. For this example: `<layout auto="1.000000"/>` specifies an automatic grid with an aspect ratio of 1.0. A specific width and heigh could be specified instead of the automatic layout. Th rest of the `<layout>` section specifies that the perimeter will hav `io` blocks, the corners will be `EMPTY` and the rest of the FPGA will be filled with `clb` blocks. `io` and `clb` blocks are defined in the `<complexblocklist>` section.

Line 32 begins the `<device>` tag which characterizes the transistors and connections of the FPGA. `<sizing>` specifies the resistance of the minimum-width nmos and pmos transistors. `<area grid_logic_tile_area>` is used as an estimate of the size of one grid tile.

Line 35 the `<chan_width_distr>` section sets the relative widths of the routing channels in various parts of the FPGA. Here, all channels are set to be distributed uniformly. Distribution options are `gaussian`, `pulse`, or `delta`.

Line 42 is the `<switchlist>` section which specifies the switches used to connect wires and pins together. Resistance, in/out capacitance, delay through the switch, and component size.

## redo segmentlist


Line 55 begins the `<complexblocklist>` section which is where the I/O and logic blocks are defined.

# Skip I/O for now...


## Logic Blocks

Line 115 is where the general purpuse *complex logic block* or *clb* definition begins. `<pb_type>` is the tag to define a physical block on the FPGA. Next, the inputs and outputs to the block are defined; here there are 10 inputs, 4 outputs, and 1 clock input. The inputs and outputs have `equivalent=true` attributes, which means that they are logically equivalent and so order doesn't matter when routing.

Line 121 defines the *basic logic element* or *BLE* that makes up the clb. The `<pb_type name="fle" num_pb="4">` attribute indicates that 4 of these BLEs called `fle` are contained in the surrounding `clb`. The next lines define 4 inputs, one output, and one clock line.

Skipping down to line 134, the core lookup-table of the BLE is defined: `<pb_type name="lut4" blif_model=".names" num_pb="1" class="lut">`.  This indicates 1 lookup-table named `lut4`; `.names` is the BLIF keyword for lookup-table. The next two lines show 4 inputs and 1 output to the LUT. The `<delay_matrix>` attribute specifies the propogation delay through the LUT's inputs to output.

Line 147 defines the flip-flop in the BLE: `<pb_type name="ff" blif_model=".latch" num_pb="1" class="flipflop">`: 1 flip-flop named `ff`; `.latch` is the BLIF keyword for flip-flop. The next lines specify the I/Os and timing parameters.

Skipping back up to line 126, a mode named `n1_lut4` for the block named `fle` is defined. A block can have multiple modes specified, but a block can only use one mode at a time. This particular block only defines one mode. The mode defines a block on line 128 named `ble4` which contains the LUT `lut4` and flip-flop `ff`.

Moving back down to line 155, `<interconnect>` indicates how the blocks are connected. The `<direct>` element means to simply wire the nets together. Line 156 wires the input of `ble4` to the input of `lut4`, line 157 wires the output of `lut4` to the D-input of `ff`. Line 161 wires the clk inputs together. Line 162 defines a mux to set the output of `ble4` to be either the direct output of `lut4` or the latched output of `ff`.